In [23]:
import numpy as np
from datetime import datetime, date, timedelta
import pandas as pd

In [24]:
class bonds:
        
    def __init__(self,
                 N : np.float,
                 tf : np.float,
                 tr : np.float,
                 d_emis : np.str,
                 d_jouis : np.str,
                 d_ech : np.str,
                 d_val : np.str):  
    
        self.N = N
        self.tf = tf
        self.tr = tr
        self.d_emis = datetime.strptime(d_emis, '%d/%m/%Y').date()
        self.d_jouis = datetime.strptime(d_jouis, '%d/%m/%Y').date()
        self.d_ech = datetime.strptime(d_ech, '%d/%m/%Y').date()
        self.d_val = datetime.strptime(d_val, '%d/%m/%Y').date()
           
        "sortie"
        
        self.dirty_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[0]
        self.coupon_couru = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[1]
        self.clean_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[2]
        self.coupon_recu = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[3]
        self.duration = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[4]
        self.sensibilite = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[5]
        self.convexite = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[6]
    
    def calculator(self,N,tf,tr,d_emis,d_jouis,d_ech,d_val): #7 parametres dont 4 dates
        
        "calcul intermediaire"
        
        # maturite residuelle et initiale
        self.mr = (self.d_ech - self.d_val).days
        self.mi = (self.d_ech - self.d_emis).days
        
        # date du dernier coupon (ddc) et date du prochain coupon (dpc)
        ddc = self.d_jouis
        dpc = ddc.replace(year = ddc.year+1)
        while dpc < self.d_val:
            dpc = dpc.replace(year= dpc.year +1)
        self.ddc = dpc.replace(year = dpc.year-1)
        self.dpc = dpc
        
        # Nombre de jours exact de l’année écoulée ???????
        self.A = 365
        
        # nombre de jours entre date de valorisation et date du prochain coupon 
        self.nj = (self.dpc - self.d_val).days 
        
        # nombre de coupon restant avant la maturité
        self.n = (self.d_ech.year - self.dpc.year) + 1
        
        # nombre de coupon deja recu avant la date de valorisation
        self.nc = (self.ddc.year - self.d_jouis.year) + 1
        
        # Date du premier coupon ??
        # d_pc = "date_jouissnce+ frequence"???

        if self.mi <= 365 : #maturite initiale inferieure a 1an

            C = N * tf * self.mi / 360
            Cr = C*self.nc
            P = (N + C) / (1+ tr*self.mr /360)
            Cc = N*tf* (self.mi-self.mr) / 360
            
            
            # Duration et sensiblite
            num = 0
            D = 0
            for i in range(1,self.n+1):
                num = num + (self.nj/self.A + i-1)*C
            num = num + (self.nj/self.A + self.n-1) * N
            D = num/ P
            S = D/(1+self.tr)
                    
            # Convexite
            Co = 0
            for i in range(1,self.n+1):
                num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
            num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
            Co = num/ (P*(1+self.tr)**2)              

        else:  #maturite initiale superieure a 1an

            if self.mr <= 365: #maturite residuelle inferieure a 1an

                    C = N*tf 
                    Cr = C*self.nc
                    P = (N + C) / (1+ tr*self.mr/360)
                    Cc = N*tf* (self.d_val-self.ddc).days / self.A
                    
                    # Duration et sensiblite
                    num = 0
                    D = 0
                    for i in range(1,self.n+1):
                        num = num + (self.nj/self.A + i-1)*C
                    num = num + (self.nj/self.A + self.n-1) * N
                    D = num/ P
                    S = D/(1+self.tr)
                    
                    # Convexite
                    Co = 0
                    for i in range(1,self.n+1):
                        num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
                    num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
                    Co = num/ (P*(1+self.tr)**2)

            else: # maturite residuelle superieure a 1an

                    C = N*tf
                    Cr = C*self.nc
                    Cc = N*tf* (self.d_val-self.ddc).days / self.A
                    s = 0
                    for i in range(1,self.n+1): # n compris
                        s = s + C/ (1+tr)**(i-1)
                    P = ( s + N/ (1+tr)**(self.n-1))  / (1+tr)**(self.nj / self.A)
                    
                    # Duration et sensiblite
                    num = 0
                    D = 0
                    for i in range(1,self.n+1):
                        num = num + C *( self.nj/self.A + i-1)
                    num = num + N*( self.nj/self.A + self.n-1)
                    D = num/ P
                    S = D/(1+self.tr)
                    
                    # Convexite
                    Co = 0
                    for i in range(1,self.n+1):
                        num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
                    num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
                    Co = num/ (P*(1+self.tr)**2)

                        
        return  (round(P),round(Cc),round(P-Cc),round(Cr), round(D,2), round(S,2), round(Co,2))
    

In [30]:
def attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift):
    
    coupon_recu1 = bonds(N,tf,tr1,d_emis,d_jouis,d_ech,d_val1).coupon_recu
    coupon_recu2 = bonds(N,tf,tr2,d_emis,d_jouis,d_ech,d_val2).coupon_recu
    coupon_couru1 = bonds(N,tf,tr1,d_emis,d_jouis,d_ech,d_val1).coupon_couru
    coupon_couru2 = bonds(N,tf,tr2,d_emis,d_jouis,d_ech,d_val2).coupon_couru
    dirty_price1 = bonds(N,tf,tr1,d_emis,d_jouis,d_ech,d_val1).dirty_price
    dirty_price2 = bonds(N,tf,tr2,d_emis,d_jouis,d_ech,d_val2).dirty_price
    
    clean_price1 = bonds(N,tf,tr1+spread1,d_emis,d_jouis,d_ech,d_val1).clean_price
    clean_price2 = bonds(N,tf,tr1+spread1,d_emis,d_jouis,d_ech,d_val2).clean_price
    clean_price3 = bonds(N,tf,tr1+spread1+shift,d_emis,d_jouis,d_ech,d_val2).clean_price
    clean_price4 = bonds(N,tf,tr2+spread1,d_emis,d_jouis,d_ech,d_val2).clean_price
    clean_price5 = bonds(N,tf,tr2+spread2,d_emis,d_jouis,d_ech,d_val2).clean_price
    coupon_recu = coupon_recu2 - coupon_recu1
        
    performance = ( dirty_price2 - dirty_price1 + coupon_recu ) / dirty_price1
    effet_coupon = (coupon_couru2 + coupon_recu - coupon_couru1) / dirty_price1
    effet_amortissement = (clean_price2 - clean_price1) / dirty_price1
    effet_niveau = (clean_price3 - clean_price2) / dirty_price1
    effet_courbe = (clean_price4 - clean_price3) / dirty_price1
    effet_spread = (clean_price5 - clean_price4) / dirty_price1
    effet_total = effet_coupon + effet_amortissement + effet_niveau + effet_courbe + effet_spread
    
    return (performance,effet_coupon,effet_amortissement,effet_niveau,effet_courbe,effet_spread)

    """
    print ("Effet coupon =", effet_coupon)
    print("Effet amortissement =", effet_amortissement)
    print("Effet niveau =", effet_niveau)
    print("Effet courbe =", effet_courbe)
    print("Effet spread =", effet_spread)
    print("Effet total =", effet_total)
    print ("performance =", performance)
    """

## Example

In [31]:
N = 100000
tf = 0.039
tr1 = 0.02463
tr2 = 0.0345
d_emis = "21/10/2014"
d_jouis = "21/10/2014"
d_ech = "21/10/2021"
d_val1 = "05/05/2017"
d_val2 = "05/05/2028"
spread1 = 0
spread2 = 0
shift = 0.002
# tr1 correspond au taux de rendement de la courbe ZC à la date de valorisation d_val1
# tr2 correspond au taux de rendement de la courbe ZC à la date de valorisation d_val1

attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift)

(0.6434505856447643,
 0.3970079381233462,
 0.14889069814777123,
 0.01851303591584479,
 0.07903891345780212,
 0.0)

## Real Data

In [15]:
import os
os.getcwd()
os.chdir("C:\\Users\\Halem\\Downloads\\final")

In [16]:
#pd.set_option("display.max_row",5)
#pd.set_option("display.max_column",6)
bond = pd.read_excel("ptfbonds.xlsx") 

In [40]:
data = pd.DataFrame( {'Short Name' : bond['Short Name'], 'Performance' : None, 
                      'Effet coupon' : None ,'Effet amortissement' : None,
                      'Effet niveau' : None , 'Effet courbe' : None, 'Effet taux' : None,
                      'Effet spread' : None  } )

In [43]:
for i in range (44): 
    
    N = 100000
    tf = bond['Cpn Rate'][i]/100
    tr1 = bond['Yield Mat'][i]/100
    tr2 = bond['YTM'][i]/100
    d_emis = bond['Date emission'][i].strftime("%d/%m/%Y")
    d_jouis = bond['Jouissance'][i].strftime("%d/%m/%Y")
    d_ech = bond['Mat. Date'][i].strftime("%d/%m/%Y")
    d_val = "08/06/2021"
    d_val2 = "09/06/2021"
    spread1 = 0
    spread2 = 0
    shift = 0.002

    data['Performance'][i] = 100*attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift)[0]
    data['Effet coupon'][i] = 100*attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift)[1]
    data['Effet amortissement'][i] = 100*attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift)[2]
    data['Effet niveau'][i] = 100*attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift)[3]
    data['Effet courbe'][i] = 100*attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift)[4]
    data['Effet spread'][i] = 100*attribution(N,tf,tr1,tr2,d_emis,d_jouis,d_ech,d_val1,d_val2,spread1,spread2,shift)[5]
    #data['Effet total'][i] = data['Effet coupon'][i] + data['Effet amortissement'][i] + data['Effet niveau'][i] + data['Effet courbe'][i] 
    data['Effet taux'][i] = data['Effet niveau'][i] + data['Effet courbe'][i] 


In [44]:
data.iloc[0:44,:]

,Short Name,Performance,Effet coupon,Effet amortissement,Effet niveau,Effet courbe,Effet taux,Effet spread
0,MA0002007096,6.86757,19.4454,-12.5817,-0.458409,0.4623,0.00389141,0
1,MA0002007641,7.29581,18.101,-10.8082,-0.671425,0.675139,0.00371363,0
2,MA0002008128,5.81064,13.4911,-7.69397,-0.122728,0.136264,0.0135362,0
3,MA0002009936,7.32563,14.6147,-7.29165,-0.637159,0.639708,0.00254864,0
4,MA0002010934,8.09149,14.8498,-6.75912,-0.967354,0.968178,0.000823981,0
5,MA0002011114,5.9644,15.2338,-9.27736,-0.155863,0.163833,0.00797024,0
6,MA0002012369,8.21251,15.9474,-7.73937,-1.09016,1.09535,0.00519122,0
7,MA0002012922,6.48742,17.2327,-10.7501,-0.354223,0.359032,0.00480846,0
8,MA0002013441,10.0757,15.9227,-5.85248,-1.88592,1.89143,0.00550433,0
9,MA0002013797,8.9086,13.3556,-4.45919,-1.43314,1.44536,0.0122281,0
